# Project Proposal

### Members:
- Zander Holoboff (95785903)
- Shuyu Li(58785809)
- Kabir Sodhi(43245943)
- Cheryl Ang (28952851)


Our Dataset:
https://archive.ics.uci.edu/dataset/186/wine+quality


# Introduction
*(Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question)*

##### This project focuses on answering a classification question.

##### Background information:
- Vinho Verde originates from the Minho region in northwestern Portugal. It is made from a blend of different grape varieties that maintains the wine's natural acidity and refreshing taste at the same time.
- Vinho Verde means "green wine" in Portuguese, but the "green" in the name does not refer to the color of the wine itself, but to the fact that the wine is best consumed within a year or two of production.
- Because red white has more vitamins in general than white wine, we will look at other variables likes sulfur dioxide, sulfate content, alcohol content, chloride content an citric acid content as predictors for the categories of either red or white wine.
##### Question:

*Can we accurately predict the color of Vinho Verde wine (either red or white) based on physiochemical and sensory variables?*

##### Description of the Dataset:
The dataset includes two categories of wines: red and white wine.  Due to privacy and logistical issues, both datasets include only the following variables:

##### These five variables are usually expressed in g/L (grams per liter):

- Fixed acidity: concentration of non-volatile acids in the wine
- Volatile acidity: concentration of volatile acids in the wine
- Citric acid: concentration of citric acid in the wine
- Residual Sugar: the amount of sugar left in the wine after fermentation.
- Chloride: concentration of salts in the wine
- Sulfate: The concentration of sulfur dioxide salts in a wine.
  
##### These two variables are usually expressed in parts per million (ppm):

- Free Sulfur Dioxide: Concentration of free sulfur dioxide in the wine as a preservative and antioxidant.
- Total Sulfur Dioxide: The total concentration of sulfur dioxide, both free and bound.

  ----------

##### Other variables:

- Density: The density of the wine, usually measured in g/ml.
- pH: The acidity or alkalinity of a wine, measured on a scale of 0 to 14, the lower the value, the higher the acidity.
- Alcohol content: The alcohol content of a wine, usually expressed as a percentage by volume.
- Quality: Sensory variables representing the quality of a wine, which can be rated on a scale from poor to excellent.

After some more research, it was found that wine makers care more about free sulfure dioxide because it is the active part of sulfur dioxide where as the bound sulfur dioxide does not react with oxygen and other wine materials. Hence, we will look at the free sulfur dioxide content because it would be interesting to see how much sulfur dioxide reacted with oxygen and other wine materials in red wine and white wine as a predictor.

### Thus, our predictors will be free sulfur dioxide content (ppm), chloride content (g/L), residual sugar content, and citric acid content.





In [44]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org



In [41]:
red_wine<-read_delim(file="data/winequality-red.csv",delim=";")|>
        rename(fixed_acidity = `fixed acidity`,
               volatile_acidity = `volatile acidity`,
               citric_acid = `citric acid`,
               residual_sugar = `residual sugar`,
               free_SO2 = `free sulfur dioxide`,
               total_SO2 = `total sulfur dioxide`
                )
#glimpse(red_wine)
white_wine<-read_delim(file="data/winequality-red.csv",delim=";")|>
        rename(fixed_acidity = `fixed acidity`,
               volatile_acidity = `volatile acidity`,
               citric_acid = `citric acid`,
               residual_sugar = `residual sugar`,
               free_SO2 = `free sulfur dioxide`,
               total_SO2 = `total sulfur dioxide`
                )
#glimpse(white_wine)

Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (12): fixed acidity, volatile acidity, citric acid, residual sugar, chlo...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1599 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (12): fixed acidity, volatile acidity, citric acid, residual sugar, chlo...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [42]:
red_wine$color<- "red" # adds  category with $
white_wine$color <- "white" # adds category with $
combined_wine<-bind_rows(red_wine,white_wine)
write_csv2(combined_wine,"combined_wine.csv")

combined_wine2<-combined_wine|>
                select(free_SO2, chlorides, residual_sugar,citric_acid,color)

combined_wine2


free_SO2,chlorides,residual_sugar,citric_acid,color
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
11,0.076,1.9,0.00,red
25,0.098,2.6,0.00,red
15,0.092,2.3,0.04,red
⋮,⋮,⋮,⋮,⋮
29,0.076,2.3,0.13,white
32,0.075,2.0,0.12,white
18,0.067,3.6,0.47,white


In [45]:
wine_split<-initial_split(data=combined_wine2, prop=0.75, strata=color)
wine_train<-training(wine_split)
wine_train

free_SO2,chlorides,residual_sugar,citric_acid,color
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
11,0.076,1.9,0.00,red
15,0.092,2.3,0.04,red
17,0.075,1.9,0.56,red
⋮,⋮,⋮,⋮,⋮
39,0.062,2.2,0.10,white
29,0.076,2.3,0.13,white
18,0.067,3.6,0.47,white
